In [1]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense


C:\Users\LOQ\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\keras\src\export\tf2onnx_lib.py:8: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):


In [4]:
import os, cv2, numpy as np

DATASET_PATH = "./dataset/images"  # ✅ FIXED PATH

print("Looking in:", os.path.abspath(DATASET_PATH))

X, y = [], []

for label, category in enumerate(["real", "fake"]):
    folder = os.path.join(DATASET_PATH, category)
    print("Checking folder:", folder)

    if not os.path.exists(folder):
        print("❌ Folder not found:", folder)
        continue

    files = os.listdir(folder)
    print(f"Found {len(files)} files in {category}")

    for file in files:
        path = os.path.join(folder, file)
        img = cv2.imread(path)

        if img is None:
            print("❌ Cannot read:", path)
            continue

        img = cv2.resize(img, (224, 224))
        X.append(img)
        y.append(label)

X = np.array(X) / 255.0
y = np.array(y)

print("FINAL X:", X.shape)
print("FINAL y:", y.shape)


Looking in: d:\Machine Learning Data\AI-media\training\dataset\images
Checking folder: ./dataset/images\real
Found 0 files in real
Checking folder: ./dataset/images\fake
Found 0 files in fake
FINAL X: (0,)
FINAL y: (0,)


In [6]:
# train_test_split already imported above; guard against empty dataset
if X.size == 0 or y.size == 0:
    print("Skipping train_test_split: No data loaded. Please add images and re-run the notebook.")
    X_train = X_test = y_train = y_test = None
else:
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42
    )
    print(X_train.shape, X_test.shape)


Skipping train_test_split: No data loaded. Please add images and re-run the notebook.


In [7]:
model = Sequential([
    Conv2D(32, (3,3), activation="relu", input_shape=(224,224,3)),
    MaxPooling2D(),

    Conv2D(64, (3,3), activation="relu"),
    MaxPooling2D(),

    Flatten(),
    Dense(128, activation="relu"),
    Dense(1, activation="sigmoid")
])

model.compile(
    optimizer="adam",
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

model.summary()


C:\Users\LOQ\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 222, 222, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 111, 111, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 109, 109, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 54, 54, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 186624)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │    23,888,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 23,907,521 (91.20 MB)

 Trainable params: 23,907,521 (91.20 MB)

 Non-trainable params: 0 (0.00 B)

In [9]:
# Guard against empty or missing training/validation data
if X_train is None or y_train is None or getattr(X_train, "size", 0) == 0 or getattr(y_train, "size", 0) == 0:
    print("Skipping training: No training data found. Please add images to the dataset and re-run the notebook.")
    history = None
else:
    # only pass validation_data if it's available and non-empty
    val_data = None
    if X_test is not None and y_test is not None and getattr(X_test, "size", 0) > 0 and getattr(y_test, "size", 0) > 0:
        val_data = (X_test, y_test)

    history = model.fit(
        X_train, y_train,
        epochs=10,
        validation_data=val_data
    )


Skipping training: No training data found. Please add images to the dataset and re-run the notebook.


In [11]:
idx = 0  # change index

if X_test is None or getattr(X_test, "size", 0) == 0:
	print("No test data available (X_test is None or empty). Please load data and run train_test_split.")
else:
	if idx < 0 or idx >= len(X_test):
		print(f"Index {idx} out of range for X_test (length {len(X_test)}).")
	elif model is None:
		print("Model is not available. Please train or load a model before predicting.")
	else:
		sample = X_test[idx].reshape(1,224,224,3)
		pred = model.predict(sample)[0][0]
		label_text = "FAKE (AI)" if pred > 0.5 else "REAL"
		confidence = pred if pred > 0.5 else 1 - pred

		plt.imshow(X_test[idx])
		plt.title(f"{label_text} | {confidence*100:.2f}%")
		plt.axis("off")


No test data available (X_test is None or empty). Please load data and run train_test_split.


In [13]:
# save in Keras native format (.keras) and ensure directory exists
save_path = "../backend/model/media_detector.keras"
save_dir = os.path.dirname(save_path)
if save_dir and not os.path.exists(save_dir):
	os.makedirs(save_dir, exist_ok=True)

model.save(save_path)
print(f"Model saved successfully at {save_path}")


Model saved successfully at ../backend/model/media_detector.keras
